Tools

Models can request to call tools that perform tasks such as fetching data from a database, searching the web, or running code. Tools are pairings of:

A schema, including the name of the tool, a description, and/or argument definitions (often a JSON schema)
A function or coroutine to execute.

In [2]:
import os
from langchain.chat_models import init_chat_model

os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")

model = init_chat_model("groq:qwen/qwen3-32b")
response = model.invoke("Why do parrots talk?")
response.content

'<think>\nOkay, so the user is asking why parrots talk. Let me start by recalling what I know about parrots. I know they\'re known for their ability to mimic human speech, but why do they do that? Maybe it\'s related to their natural behavior? I remember that in the wild, parrots communicate with each other through sounds. So in captivity, maybe they imitate human speech as a way of social interaction. But why would they specifically talk to humans?\n\nI should think about the evolutionary aspect. Parrots are social animals, so communication is crucial for their survival. In the wild, they use calls to bond, warn each other of danger, and find mates. When they\'re kept as pets, they might treat their human caregivers as part of their social group, leading them to mimic human sounds to communicate. But not all parrots talk, right? Some species are better at it than others. What\'s the difference between species? Maybe the structure of their vocal organs plays a role. They have a syrinx,

In [3]:
from langchain.tools import tool

@tool
def get_weather(location:str)->str:
    """Get the weather at a location"""
    return f"It's sunny in {location}"


model_with_tools=model.bind_tools([get_weather])

In [5]:
response = model_with_tools.invoke("What's the weather like in Boston?")
print(response)
for tool_call in response.tool_calls:
    # View tool calls made by the model
    print(f"Tool: {tool_call['name']}")
    print(f"Args: {tool_call['args']}")

content='' additional_kwargs={'reasoning_content': 'Okay, the user is asking about the weather in Boston. I need to use the get_weather function. Let me check the function parameters. It requires a location, which in this case is Boston. I should structure the tool call with the location parameter set to "Boston". Make sure the JSON is correctly formatted. No other parameters are needed. Just call get_weather with location: Boston.\n', 'tool_calls': [{'id': 'cpnebcd67', 'function': {'arguments': '{"location":"Boston"}', 'name': 'get_weather'}, 'type': 'function'}]} response_metadata={'token_usage': {'completion_tokens': 102, 'prompt_tokens': 154, 'total_tokens': 256, 'completion_time': 0.169951611, 'completion_tokens_details': {'reasoning_tokens': 78}, 'prompt_time': 0.006961966, 'prompt_tokens_details': None, 'queue_time': 0.158449773, 'total_time': 0.176913577}, 'model_name': 'qwen/qwen3-32b', 'system_fingerprint': 'fp_2bfcc54d36', 'service_tier': 'on_demand', 'finish_reason': 'tool_

# Tool Execution Loops

In [8]:
messages = [{"role": "user", "content": "What's the weather in Boston?"}]
ai_msg = model_with_tools.invoke(messages)
messages.append(ai_msg)
messages

[{'role': 'user', 'content': "What's the weather in Boston?"},
 AIMessage(content='', additional_kwargs={'reasoning_content': 'Okay, the user is asking for the weather in Boston. Let me check what tools I have available. There\'s a function called get_weather that takes a location parameter. Since the user specified Boston, I need to call that function with "Boston" as the location. I\'ll make sure to format the tool call correctly within the XML tags. No other functions are available, so this should be straightforward. Let me structure the JSON with the function name and the arguments as required.\n', 'tool_calls': [{'id': 'd6yjh23cn', 'function': {'arguments': '{"location":"Boston"}', 'name': 'get_weather'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 120, 'prompt_tokens': 153, 'total_tokens': 273, 'completion_time': 0.201464961, 'completion_tokens_details': {'reasoning_tokens': 96}, 'prompt_time': 0.006564692, 'prompt_tokens_details': None, 'queue_

In [9]:
ai_msg.tool_calls

[{'name': 'get_weather',
  'args': {'location': 'Boston'},
  'id': 'd6yjh23cn',
  'type': 'tool_call'}]

In [12]:
for tool_call in ai_msg.tool_calls:
    # Execute the tool with the generated arguments
    tool_result = get_weather.invoke(tool_call)
    messages.append(tool_result)

In [13]:
# Step 3: Pass results back to model for final response
final_response = model_with_tools.invoke(messages)
print(final_response.text)

The weather in Boston is sunny right now! If you're there, it's a great day to enjoy outdoor activities. 😊
